In [1]:
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

import pandas as pd
import numpy as np
import sys
import h5py
import pickle
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from scipy.sparse import csr_matrix, hstack
from sklearn.cross_validation import KFold, train_test_split
from keras.layers.advanced_activations import PReLU
from keras.models import Sequential
from keras.models import save_model, load_model
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
import matplotlib.patches as mpatches


%matplotlib inline

/home/coder.chenshicheng/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
ids = test['id']
test['loss'] = np.nan
joined = pd.concat([train, test])
f_num = [f for f in train.columns if 'cont' in f]
f_cat = [f for f in train.columns if 'cat' in f]
for column in f_cat:
    if (train[column].nunique() != test[column].nunique()
        or (train[column].unique() != test[column].unique()).any()):
        set_train = set(train[column].unique())
        set_test = set(test[column].unique())
        
        remove = ((set_train - set_test)|(set_test - set_train))
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x
        myfilter = lambda x: np.nan if x in remove else x
        joined[column] = joined[column].apply(filter_cat, 1)

    #joined[column] = pd.factorize(joined[column].values, sort=True)[0]

train = joined[joined['loss'].notnull()]
test = joined[joined['loss'].isnull()]

shift = 200
#train_y = np.log(train['loss'] + shift)
train_y=train['loss']
train_x = train.drop(['loss', 'id'], 1)
test = test.drop(['loss', 'id'], 1)

In [3]:
scaler = StandardScaler()
contx = scaler.fit_transform(train_x[f_num])
catx = pd.get_dummies(data=train_x[f_cat],columns=f_cat,dummy_na=False).values

print (catx.shape,contx.shape)

train_x = np.concatenate((contx,catx),axis=1)


test = pd.get_dummies(data=test,columns=f_cat).values
print (train_x.shape)
print (test.shape)
print (train_y.shape)

(188318, 1065) (188318, 14)
(188318, 1079)
(125546, 1079)
(188318,)


In [62]:
import keras.backend as K

def mae_score(y_true, y_pred):
    return K.mean(K.abs(K.exp(y_true)-K.exp(y_pred)), axis=-1)

from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', 
                             verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

In [4]:
# http://cs231n.github.io/neural-networks-3/

import numpy as np
import scipy.sparse as sp

from keras import backend as K
from keras.callbacks import Callback
from keras.models import load_model

import sys
import warnings
class ExponentialMovingAverage(Callback):
    """create a copy of trainable weights which gets updated at every
       batch using exponential weight decay. The moving average weights along
       with the other states of original model(except original model trainable
       weights) will be saved at every epoch if save_mv_ave_model is True.
       If both save_mv_ave_model and save_best_only are True, the latest
       best moving average model according to the quantity monitored
       will not be overwritten. Of course, save_best_only can be True
       only if there is a validation set.
       This is equivalent to save_best_only mode of ModelCheckpoint
       callback with similar code. custom_objects is a dictionary
       holding name and Class implementation for custom layers.
       At end of every batch, the update is as follows:
       mv_weight -= (1 - decay) * (mv_weight - weight)
       where weight and mv_weight is the ordinal model weight and the moving
       averaged weight respectively. At the end of the training, the moving
       averaged weights are transferred to the original model.
       """
    def __init__(self, decay=0.999, filepath='model/model{epoch:02d}-{val_loss:.2f}.hdf5',
                 save_mv_ave_model=False, verbose=0,
                 save_best_only=False, monitor='val_loss', mode='auto',
                 save_weights_only=False, custom_objects={}):
        self.decay = decay
        self.filepath = filepath
        self.verbose = verbose
        self.save_mv_ave_model = save_mv_ave_model
        self.save_weights_only = save_weights_only
        self.save_best_only = save_best_only
        self.monitor = monitor
        self.custom_objects = custom_objects  # dictionary of custom layers
        self.sym_trainable_weights = None  # trainable weights of model
        self.mv_trainable_weights_vals = None  # moving averaged values
        self.epochs = 0

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('ModelCheckpoint mode %s is unknown, '
                          'fallback to auto mode.' % (mode),
                          RuntimeWarning)
            mode = 'auto'

        if mode == 'min':
            self.monitor_op = np.less
            self.best = np.Inf
        elif mode == 'max':
            self.monitor_op = np.greater
            self.best = -np.Inf
        else:
            if 'acc' in self.monitor:
                self.monitor_op = np.greater
                self.best = -np.Inf
            else:
                self.monitor_op = np.less
                self.best = np.Inf

    def on_train_begin(self, logs={}):
        self.sym_trainable_weights = self.model.trainable_weights
        # Initialize moving averaged weights using original model values
        self.mv_trainable_weights_vals = {x.name: K.get_value(x).copy() for x in
                                          self.sym_trainable_weights}
        if self.verbose:
            print('Created a copy of model weights to initialize moving averaged weights.')
    def on_train_end(self, logs={}):
        '''old = K.get_value(self.model.trainable_weights[0])
        name = self.model.trainable_weights[0].name
        assert((old != self.mv_trainable_weights_vals[name]).any())'''
        
        for weight in self.sym_trainable_weights:
            K.set_value(weight, self.mv_trainable_weights_vals[weight.name])

        '''old = K.get_value(self.model.trainable_weights[0])
        name = self.model.trainable_weights[0].name
        assert((old == self.mv_trainable_weights_vals[name]).all())'''
            
    def on_batch_end(self, batch, logs={}):
        if(self.epochs + 5 > self.params['epochs']):
            for weight in self.sym_trainable_weights:
                old_val = self.mv_trainable_weights_vals[weight.name].copy()
                self.mv_trainable_weights_vals[weight.name] = \
                    old_val * self.decay + (1.0 - self.decay) * K.get_value(weight).copy()
            #assert((old_val == self.mv_trainable_weights_vals[weight.name]).all())
            #assert((old_val == K.get_value(weight)).all())

    def on_epoch_end(self, epoch, logs={}):
        self.epochs += 1
        if(self.epochs + 5 > self.params['epochs']):
            filepath = self.filepath.format(epoch=epoch + 1, **logs)
            self.model.save(filepath, overwrite=True)

In [5]:
from keras import regularizers
from keras.initializers import he_normal
from keras.regularizers import l2
from keras.activations import selu
from keras.optimizers import Adam
def hyper_model(seed = None):
    model = Sequential()
    model.add(Dense(437, input_dim=train_x.shape[1], kernel_initializer=he_normal(seed = seed)
                    ,kernel_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.536))
    
    model.add(Dense(182, kernel_initializer=he_normal(seed = seed),kernel_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Dense(73, kernel_initializer=he_normal(seed = seed),kernel_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.233))
    
    model.add(Dense(1, kernel_initializer=he_normal(seed = seed),kernel_regularizer=l2(0.002)))
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    #model.compile(optimizer='adadelta',loss = 'mae',metrics = [mae_score])
    model.compile(optimizer='adadelta',loss = 'mae')
    return model

In [6]:
nnmodel={}
nnmodel.clear()
def cross_validate_mlp(mlp_func, nfolds=10,nbags=5):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nbags,))
    stack_train = np.zeros((nbags,len(train_y)))
    stack_test = np.zeros((nbags,len(test)))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        for bag in range(nbags):
            nnmodel['nn%d',k*10+bag*1] = mlp_func(seed = k*10+bag*1)
            early_stopping = EarlyStopping(monitor='val_loss', patience=5)
            fit = nnmodel['nn%d',k*10+bag*1].fit(xtr, ytr, validation_split=0.2, batch_size=128,
                          epochs=30, verbose=1, callbacks=[ExponentialMovingAverage()])
            pred = nnmodel['nn%d',k*10+bag*1].predict(xte, batch_size=1024)
            #score = mean_absolute_error(np.exp(yte), np.exp(pred))
            score = mean_absolute_error(yte, pred)
            val_scores[bag] += score
            #stack_train[bag][test_index] = pred[:,0]
            print ("nfold:{},bag:{}".format(k,bag),score)
    for bag in range(nbags):
        val_scores[bag] = val_scores[bag] / float(nfolds)
    
    
    return val_scores

cv_score = cross_validate_mlp(hyper_model)
print ("CV score for the final model:", cv_score)

Train on 135588 samples, validate on 33898 samples
Epoch 1/30
135588/135588 [==============================] - 11s 80us/step - loss: 2141.2406 - val_loss: 1227.3853
Epoch 2/30
135588/135588 [==============================] - 10s 76us/step - loss: 1377.5925 - val_loss: 1180.3451
Epoch 3/30
135588/135588 [==============================] - 10s 76us/step - loss: 1353.0381 - val_loss: 1166.2284
Epoch 4/30
135588/135588 [==============================] - 10s 76us/step - loss: 1340.3170 - val_loss: 1158.8036
Epoch 5/30
135588/135588 [==============================] - 10s 74us/step - loss: 1330.2459 - val_loss: 1155.6613
Epoch 6/30
135588/135588 [==============================] - 10s 74us/step - loss: 1321.8207 - val_loss: 1152.8829
Epoch 7/30
135588/135588 [==============================] - 10s 75us/step - loss: 1319.1778 - val_loss: 1153.1723
Epoch 8/30
135588/135588 [==============================] - 10s 75us/step - loss: 1307.5080 - val_loss: 1150.6116
Epoch 9/30
135588/135588 [===========

135588/135588 [==============================] - 10s 76us/step - loss: 1236.6628 - val_loss: 1142.4172
Epoch 11/30
135588/135588 [==============================] - 10s 76us/step - loss: 1233.7533 - val_loss: 1140.2531
Epoch 12/30
135588/135588 [==============================] - 10s 77us/step - loss: 1229.3597 - val_loss: 1139.7299
Epoch 13/30
135588/135588 [==============================] - 10s 77us/step - loss: 1228.2212 - val_loss: 1139.0084
Epoch 14/30
135588/135588 [==============================] - 10s 76us/step - loss: 1221.2148 - val_loss: 1142.1605
Epoch 15/30
135588/135588 [==============================] - 10s 76us/step - loss: 1221.4302 - val_loss: 1138.1203
Epoch 16/30
135588/135588 [==============================] - 10s 76us/step - loss: 1217.7132 - val_loss: 1139.1325
Epoch 17/30
135588/135588 [==============================] - 10s 76us/step - loss: 1214.1610 - val_loss: 1138.3226
Epoch 18/30
135588/135588 [==============================] - 10s 77us/step - loss: 1210.8970

135588/135588 [==============================] - 10s 77us/step - loss: 1205.5047 - val_loss: 1136.1903
Epoch 21/30
135588/135588 [==============================] - 10s 77us/step - loss: 1204.2275 - val_loss: 1138.6387
Epoch 22/30
135588/135588 [==============================] - 10s 77us/step - loss: 1197.6086 - val_loss: 1137.2679
Epoch 23/30
135588/135588 [==============================] - 10s 77us/step - loss: 1195.8666 - val_loss: 1138.0406
Epoch 24/30
135588/135588 [==============================] - 10s 76us/step - loss: 1197.2501 - val_loss: 1136.2352
Epoch 25/30
135588/135588 [==============================] - 10s 77us/step - loss: 1195.3602 - val_loss: 1136.4173
Epoch 26/30
135588/135588 [==============================] - 11s 81us/step - loss: 1189.4139 - val_loss: 1137.3516
Epoch 27/30
135588/135588 [==============================] - 19s 137us/step - loss: 1188.7968 - val_loss: 1135.0059
Epoch 28/30
135588/135588 [==============================] - 19s 137us/step - loss: 1181.65

135588/135588 [==============================] - 20s 147us/step - loss: 1188.4280 - val_loss: 1140.5027
nfold:1,bag:1 1161.12787394
Train on 135588 samples, validate on 33898 samples
Epoch 1/30
135588/135588 [==============================] - 11s 81us/step - loss: 2102.7812 - val_loss: 1202.3455
Epoch 2/30
135588/135588 [==============================] - 11s 78us/step - loss: 1368.7018 - val_loss: 1178.5006
Epoch 3/30
135588/135588 [==============================] - 10s 77us/step - loss: 1342.8355 - val_loss: 1164.6004
Epoch 4/30
135588/135588 [==============================] - 11s 78us/step - loss: 1329.0768 - val_loss: 1167.7070
Epoch 5/30
135588/135588 [==============================] - 10s 76us/step - loss: 1317.4153 - val_loss: 1161.0539
Epoch 6/30
135588/135588 [==============================] - 10s 77us/step - loss: 1311.6860 - val_loss: 1155.1814
Epoch 7/30
135588/135588 [==============================] - 10s 77us/step - loss: 1307.4906 - val_loss: 1156.7080
Epoch 8/30
135588/1

135588/135588 [==============================] - 9s 66us/step - loss: 1280.9761 - val_loss: 1151.9654
Epoch 10/30
135588/135588 [==============================] - 9s 65us/step - loss: 1270.0743 - val_loss: 1148.6066
Epoch 11/30
135588/135588 [==============================] - 9s 67us/step - loss: 1266.6186 - val_loss: 1149.1808
Epoch 12/30
135588/135588 [==============================] - 9s 65us/step - loss: 1261.7751 - val_loss: 1148.9935
Epoch 13/30
135588/135588 [==============================] - 9s 66us/step - loss: 1252.8905 - val_loss: 1145.2222
Epoch 14/30
135588/135588 [==============================] - 9s 68us/step - loss: 1248.4711 - val_loss: 1145.8444
Epoch 15/30
135588/135588 [==============================] - 9s 65us/step - loss: 1243.4477 - val_loss: 1143.7418
Epoch 16/30
135588/135588 [==============================] - 9s 67us/step - loss: 1238.0203 - val_loss: 1144.6408
Epoch 17/30
135588/135588 [==============================] - 9s 69us/step - loss: 1234.5789 - val_lo

...

135588/135588 [==============================] - 11s 79us/step - loss: 1252.7494 - val_loss: 1140.3385
Epoch 16/30
135588/135588 [==============================] - 11s 79us/step - loss: 1251.6632 - val_loss: 1138.7266
Epoch 17/30
135588/135588 [==============================] - 11s 81us/step - loss: 1242.9754 - val_loss: 1137.6287
Epoch 18/30
135588/135588 [==============================] - 11s 80us/step - loss: 1238.5133 - val_loss: 1136.6349
Epoch 19/30
135588/135588 [==============================] - 11s 80us/step - loss: 1234.0214 - val_loss: 1142.3731
Epoch 20/30
135588/135588 [==============================] - 11s 81us/step - loss: 1230.8911 - val_loss: 1137.1442
Epoch 21/30
135588/135588 [==============================] - 11s 80us/step - loss: 1225.0861 - val_loss: 1135.3693
Epoch 22/30
135588/135588 [==============================] - 11s 78us/step - loss: 1221.2549 - val_loss: 1136.8301
Epoch 23/30
135588/135588 [==============================] - 11s 81us/step - loss: 1217.2063

135589/135589 [==============================] - 11s 84us/step - loss: 1209.2536 - val_loss: 1141.3799
Epoch 26/30
135589/135589 [==============================] - 15s 111us/step - loss: 1204.2124 - val_loss: 1142.6990
Epoch 27/30
135589/135589 [==============================] - 27s 202us/step - loss: 1198.0131 - val_loss: 1140.7735
Epoch 28/30
135589/135589 [==============================] - 27s 199us/step - loss: 1193.8403 - val_loss: 1140.8204
Epoch 29/30
135589/135589 [==============================] - 28s 203us/step - loss: 1194.1936 - val_loss: 1140.4732
Epoch 30/30
135589/135589 [==============================] - 26s 195us/step - loss: 1190.3474 - val_loss: 1142.4468
nfold:8,bag:1 1130.88282786
Train on 135589 samples, validate on 33898 samples
Epoch 1/30
135589/135589 [==============================] - 12s 90us/step - loss: 2327.7298 - val_loss: 1280.7319
Epoch 2/30
135589/135589 [==============================] - 11s 82us/step - loss: 1412.1334 - val_loss: 1188.9560
Epoch 3/30

135589/135589 [==============================] - 11s 81us/step - loss: 1379.6914 - val_loss: 1169.2786
Epoch 5/30
135589/135589 [==============================] - 11s 81us/step - loss: 1364.2350 - val_loss: 1164.8502
Epoch 6/30
135589/135589 [==============================] - 11s 81us/step - loss: 1356.1164 - val_loss: 1160.4845
Epoch 7/30
135589/135589 [==============================] - 11s 80us/step - loss: 1342.7500 - val_loss: 1155.5586
Epoch 8/30
135589/135589 [==============================] - 11s 81us/step - loss: 1334.1648 - val_loss: 1152.3382
Epoch 9/30
135589/135589 [==============================] - 11s 81us/step - loss: 1329.0797 - val_loss: 1150.6409
Epoch 10/30
135589/135589 [==============================] - 11s 81us/step - loss: 1315.7353 - val_loss: 1150.3249
Epoch 11/30
135589/135589 [==============================] - 11s 81us/step - loss: 1308.3846 - val_loss: 1148.4464
Epoch 12/30
135589/135589 [==============================] - 11s 80us/step - loss: 1295.5100 - va

135589/135589 [==============================] - 11s 83us/step - loss: 1230.1191 - val_loss: 1146.2395
Epoch 15/30
135589/135589 [==============================] - 11s 83us/step - loss: 1229.5781 - val_loss: 1148.5024
Epoch 16/30
135589/135589 [==============================] - 11s 84us/step - loss: 1224.9700 - val_loss: 1144.6198
Epoch 17/30
135589/135589 [==============================] - 12s 85us/step - loss: 1218.3857 - val_loss: 1144.4822
Epoch 18/30
135589/135589 [==============================] - 11s 83us/step - loss: 1219.2257 - val_loss: 1145.8661
Epoch 19/30
135589/135589 [==============================] - 11s 83us/step - loss: 1215.4330 - val_loss: 1148.0471
Epoch 20/30
135589/135589 [==============================] - 11s 83us/step - loss: 1208.7457 - val_loss: 1142.8046
Epoch 21/30
135589/135589 [==============================] - 11s 83us/step - loss: 1206.3806 - val_loss: 1143.1983
Epoch 22/30
135589/135589 [==============================] - 11s 83us/step - loss: 1206.7799

135589/135589 [==============================] - 11s 83us/step - loss: 1212.6539 - val_loss: 1144.7662
Epoch 25/30
135589/135589 [==============================] - 11s 84us/step - loss: 1209.2545 - val_loss: 1143.2628
Epoch 26/30
135589/135589 [==============================] - 16s 117us/step - loss: 1206.3184 - val_loss: 1143.6578
Epoch 27/30
135589/135589 [==============================] - 28s 210us/step - loss: 1202.2124 - val_loss: 1144.2421
Epoch 28/30
135589/135589 [==============================] - 30s 219us/step - loss: 1197.8196 - val_loss: 1144.5040
Epoch 29/30
135589/135589 [==============================] - 29s 212us/step - loss: 1194.9373 - val_loss: 1144.1771
Epoch 30/30
135589/135589 [==============================] - 29s 215us/step - loss: 1192.3850 - val_loss: 1144.3658
nfold:9,bag:3 1136.60712289
Train on 135589 samples, validate on 33898 samples
Epoch 1/30
135589/135589 [==============================] - 12s 92us/step - loss: 2160.3822 - val_loss: 1256.4578
Epoch 2/3

Though this model is not adapted for mere 30 epochs of training, nor for 3-fold CV (I used 5-fold on Kaggle), even though this is a single unbagged model which has been cross-validated on three folds only, we see a very good score:
`CV = 1150` (your score may vary a little).

By the way, this single model, bagged, 5-fold CVed, scored 1116.28 on Kaggle LB.

As we see, this model is considerably better than any other models we had so far. We now take it as the second part of our final ensemble.

In [ ]:
'l3-nn': {
        'predictions': l2_predictions,
        'n_bags': 4,
        'model': Keras(nn_lr, lambda: {'l1': 1e-5, 'l2': 1e-5, 'n_epoch': 30, 'batch_size': 128, 'optimizer': SGD(3e-2, momentum=0.8, nesterov=True, decay=3e-5), 'callbacks': [ExponentialMovingAverage(save_mv_ave_model=False)]}),
    },
     'optimizer': SGD(1e-4, momentum=0.9, nesterov=True, decay=5e-5)
            'optimizer': SGD(1e-5, momentum=0.9, nesterov=True, decay=5e-5)

In [ ]:
{'hidden1_dropout': 0.5368421052631579, 'hidden1_units': 437, 'hidden2_dropout': 0.40000000000000002, 
 'hidden2_units': 182, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 73, 'optimizer': 'adadelta',
 'wdecay': 0.0020300000000000001}

In [251]:
def cross_validate_mlp(mlp_func, nfolds=3):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nfolds,))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        mlp = mlp_func()
        early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        fit = mlp.fit(xtr, ytr, validation_split=0.1, batch_size=128, 
                      epochs=10, verbose=0, callbacks=[early_stopping])
        pred = mlp.predict(xte, batch_size=512)
        score = mean_absolute_error(np.exp(yte), np.exp(pred))
        val_scores[k] += score
        print ('Fold {}, MAE: {}'.format(k, score))
    avg_score = np.sum(val_scores) / float(nfolds)
    print ('{}-fold CV score: {}'.format(nfolds, avg_score))
    return avg_score

In [252]:
# VERSION 4. Insights:
# – why not to test 4-layer architectures?
# — we need to introduce new optimizers
# — adding batch normalization (https://arxiv.org/abs/1502.03167)

# Describing the search space
space = {'hidden1_dropout': hp.choice('hidden1_dropout', np.linspace(0.4,0.6,20)),
        'hidden2_dropout': hp.choice('hidden2_dropout', np.linspace(0.2,0.5,10)),
        'hidden3_dropout': hp.choice('hidden3_dropout', np.linspace(0.1,0.5,10)),
         'hidden1_units': hp.choice('hidden1_units', np.linspace(300,550,30,dtype='int32')),
         'hidden2_units': hp.choice('hidden2_units', np.linspace(100,300,30,dtype='int32')),
         'hidden3_units': hp.choice('hidden3_units', np.linspace(20,80,30,dtype='int32')),
         'optimizer': hp.choice('optimizer', ['adadelta','adam']),
         'wdecay':hp.choice('wdecay', np.linspace(0.0001,0.01,1000)),
        }

# Implementing a function to minimize
def hyperopt_search(params):
    print ('Model Testing:', params)
    def mlp_model():
        model = Sequential()
        model.add(Dense(params['hidden1_units'], input_dim=train_x.shape[1], kernel_initializer='he_normal',
                        kernel_regularizer=l2(params['wdecay'])))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden1_dropout']))
        
        model.add(Dense(params['hidden2_units'], kernel_initializer='he_normal',
                        kernel_regularizer=l2(params['wdecay'])))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden2_dropout']))

        model.add(Dense(params['hidden3_units'], kernel_initializer='he_normal',
                        kernel_regularizer=l2(params['wdecay']))) 
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden3_dropout']))
        
        model.add(Dense(1, kernel_initializer='he_normal',kernel_regularizer=l2(params['wdecay'])))
        model.compile(loss=mae_score,metrics=[mae_score], optimizer=params['optimizer'])
        return model
    
    cv_score = cross_validate_mlp(mlp_model)
    return {'loss': cv_score, 'status': STATUS_OK}


trials = Trials()

# UNCOMMENT THE NEXT LINE TO LAUNCH HYPEROPT:
best = fmin(hyperopt_search, space, algo=tpe.suggest, max_evals = 100, trials=trials)

Model Testing: {'hidden1_dropout': 0.59999999999999998, 'hidden1_units': 463, 'hidden2_dropout': 0.46666666666666667, 'hidden2_units': 155, 'hidden3_dropout': 0.18888888888888888, 'hidden3_units': 20, 'optimizer': 'adam', 'wdecay': 0.0012891891891891893}
Fold 0, MAE: 1664.3154518123333
Fold 1, MAE: 1789.6478768193072
Fold 2, MAE: 2230.6679431902985
3-fold CV score: 1894.877090607313
Model Testing: {'hidden1_dropout': 0.50526315789473686, 'hidden1_units': 308, 'hidden2_dropout': 0.20000000000000001, 'hidden2_units': 244, 'hidden3_dropout': 0.41111111111111109, 'hidden3_units': 28, 'optimizer': 'adadelta', 'wdecay': 0.0075522522522522527}
Fold 0, MAE: 2663.589775993532
Fold 1, MAE: 2550.3793884253055
Fold 2, MAE: 2698.309167392798
3-fold CV score: 2637.426110603878
Model Testing: {'hidden1_dropout': 0.59999999999999998, 'hidden1_units': 394, 'hidden2_dropout': 0.23333333333333334, 'hidden2_units': 120, 'hidden3_dropout': 0.18888888888888888, 'hidden3_units': 65, 'optimizer': 'adam', 'wde

KeyboardInterrupt: 